In [1]:
%load_ext watermark
%watermark

ModuleNotFoundError: No module named 'watermark'

In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [3]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [4]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [5]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
print(file_path)

time       = []
T_aux_avg  = []
r2_v2_rlim      = []

for i,j in enumerate(file_path):
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])

('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC03_RF03.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC04_RF04.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC05_RF05.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC06_RF06.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_tem

In [345]:
def tanh_plus(x,K,theta,x0,a,b):
    return K*(1+tanh(theta*(x-x0))) + a*(x)+b
def tanh_plus_to10(x,K,theta,x0,a,b):
    return 10**(K*(1+tanh(theta*(x-x0))) + a*(x)+b)
def gen_logi(x,A,K,C,B,M,nu):
    return A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu)
def gen_logi_to10(x,A,K,C,B,M,nu):
    return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )

In [318]:
popt

array([ 1.63532814e+03,  1.40758340e+01,  6.31322873e+01,  8.72241875e-01,
       -3.32494371e+03])

In [375]:
# OK pour la temperature
to_plot = 1
end_fit = 125500

print(file_path[to_plot])
# test plot
index_max = 1900
figure(1,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
plot(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')

p0 = [1.8, 60, 63.45, 0.002,4e-3] # 124850
popt, pcov = curve_fit(tanh_plus_to10,
                       time[to_plot][:end_fit]*1e3, T_aux_avg[to_plot][:end_fit],
                       p0, maxfev= 50000)
print(popt)
plot(time[to_plot][:end_fit]*1e3,tanh_plus_to10(time[to_plot][:end_fit]*1e3,*popt),color='k')

plot(time[to_plot][end_fit:]*1e3,tanh_plus_to10(time[to_plot][end_fit:]*1e3,
                                            1635.3,
                                            14.075,
                                            63.132,
                                            0.87,
                                            -3324.94),
     color='g',ls=':')


p0 = [5e-3,500,1,2,65,2] # 124850
popt, pcov = curve_fit(gen_logi_to10,
                       time[to_plot][:end_fit]*1e3, T_aux_avg[to_plot][:end_fit],
                       maxfev= 50000)
plot(time[to_plot][:end_fit]*1e3,gen_logi_to10(time[to_plot][:end_fit]*1e3,*popt),color='xkcd:azul')

vlines(time[to_plot][end_fit]*1e3,0,50)
#b, a = signal.butter(8, 0.125)
#y = signal.filtfilt(b, a, T_aux_avg[to_plot], padlen=150)
#semilogy(time[to_plot]*1e3,y,color='b')
xlabel('time [ms]')
ylabel('T [K]')
#xlim(61,64)
#ylim(-1,20)
grid()
tight_layout()

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
[ 1.63532814e+03  1.40758340e+01  6.31322873e+01  8.72241875e-01
 -3.32494371e+03]


/tmp/ipykernel_18528/1091199992.py:8: RuntimeWarning: overflow encountered in exp
  return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )
/tmp/ipykernel_18528/1091199992.py:8: RuntimeWarning: overflow encountered in power
  return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )
/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [349]:
fit_param

(1635.3281361954287,
 14.075833960215585,
 63.132287259238865,
 0.8722418754283464,
 -3324.9437115202995)

In [338]:
figure(11,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
semilogy(time[to_plot]*1e3,gen_logi(time[to_plot]*1e3,5e-3,500,1,2,65,2)+2e-4*time[to_plot]*1e3,color='g')
vlines(time[to_plot][124850]*1e3,1e-3,100)
xlabel('time [ms]')
ylabel('T [K]')
# xlim(1,1.1)
grid()
tight_layout()

In [129]:
# OK pour la temperature
x = linspace(-10,10,1000)

print(file_path[to_plot])
# test plot
index_max = 1900
figure(1111,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
plot(x,tanh_plus(x,1,1,0,0.1,0),color='r')
#b, a = signal.butter(8, 0.125)
#y = signal.filtfilt(b, a, T_aux_avg[to_plot], padlen=150)
#semilogy(time[to_plot]*1e3,y,color='b')
xlabel('time [ms]')
ylabel('T [K]')
# xlim(1,1.1)
grid()
tight_layout()

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz


In [376]:
# test plot r² and v²

figure(2,clear='True')
for k in range(3):
    plot(time[to_plot]*1e3,r2_v2_rlim[to_plot][k][:len(time[to_plot])],label=str(k))
xlabel('time [ms]')
ylabel('R² [m²]')
# ylim(0,100)
grid()

legend()
tight_layout()

figure(3,clear='True')
for k in range(3):
    semilogy(time[to_plot]*1e3,r2_v2_rlim[to_plot][k+3][:len(time[to_plot])],label=str(k))
xlabel('time [ms]')
ylabel('V² [m/s ²]')
grid()
legend()
tight_layout()

figure(21,clear='True')
for k in range(3):
    plot(time[to_plot]*1e3,r2_v2_rlim[to_plot][k+6][:len(time[to_plot])],label=str(k))
xlabel('time [ms]')
ylabel('R_lim [m]')
grid()
tight_layout()

b, a = signal.butter(8, 0.125)
R_lim_x_filt = signal.filtfilt(b, a, r2_v2_rlim[to_plot][6][:len(time[to_plot])], padlen=150)
figure(22,clear='True')
plot(T_aux_avg[to_plot],r2_v2_rlim[to_plot][6][:len(time[to_plot])],label=str(k),marker='+',ls='')
a = 2.5e-3
b = 0.085
plot(linspace(0,40,1000),a*linspace(0,40,1000)+b,label=str(k),ls='-')
a = 2.7e-3
b = 0.075
plot(linspace(0,40,1000),a*linspace(0,40,1000)+b,label=str(k),ls='-')
xlabel('Temperature [K]')
ylabel('R_lim [m]')
xlim(-0.5,42)
ylim(0.02,0.22)
grid()
tight_layout()

In [ ]:
r2_v2_rlim

In [377]:
# trim data from dirty edges
start = 50
end = len(time[to_plot])-500
time_good = time[to_plot][start:end]
T_good = T_aux_avg[to_plot][start:end]
#cloud_size   = sqrt(r2_v2[0,start:end]+r2_v2[1,start:end]+r2_v2[2,start:end])
cloud_size_edge = sqrt(r2_v2_rlim[to_plot][6][start:end]+r2_v2_rlim[to_plot][7][start:end]+r2_v2_rlim[to_plot][8][start:end])
#crystal_size = sqrt(r2_v2[0,5]+r2_v2[1,5]+r2_v2[2,5])
# filter data
b, a = signal.butter(8, 0.125)
y = signal.filtfilt(b, a, T_good, padlen=150)
print(len(y),len(cloud_size_edge))

211244 211244


In [378]:
time_step = 48//4 # 488 # 488
y_diff = (y[time_step:]-y[:-time_step])
cloud_size_edge_diff = (cloud_size_edge[time_step:]+cloud_size_edge[:-time_step])/2
print(len(y_diff),len(cloud_size_edge_diff))

211232 211232


In [379]:
# Heating by differentiating
# between points spaced by 50 RF periods
N_period = (time_good[time_step]-time_good[0])*(Omega/2/pi)
time_good_diff = time_good[time_step//2:-time_step//2]*1e3
cloud_size_edge = cloud_size_edge[time_step:-time_step]

b, a = signal.butter(8, 0.05)
y_diff_filt = signal.filtfilt(b, a, y_diff, padlen=150)

figure(141,clear='True')
plot(time_good[:-time_step]*1e3,
     y_diff/N_period)
plot(time_good[:-time_step]*1e3,
     y_diff_filt/N_period)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('time [ms]')
ylabel('Heating rate [K/period]')
tight_layout()

figure(142,clear='True')
plot(cloud_size_edge_diff,
     y_diff/N_period)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('Cloud size []')
ylabel('Heating rate [K/period]')
tight_layout()

R = 88*1e-6
V = 4/3*pi*R**3
rho = 256/V
aws = (1/rho)**(1/3)
temperature_gamma = C_e**2 / (4*pi*eps0*aws*kb*y)

figure(143,clear='True')
plot(1/temperature_gamma[:-time_step],
     y_diff/N_period)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('Temperature [1/$\Gamma$]')
ylabel('Heating rate [K/period]')
tight_layout()

#figure(144,clear='True')
#loglog(1/temperature_gamma[:-time_step],
#     (Ekin[:len(y_diff)]/(y_diff/N_period))**-1)
#grid()
#xlim(35.3,37.4)
#ylim(-1,75)
#xlabel('Temperature [1/$\Gamma$]')
#ylabel('Heating rate [K/period]')
#tight_layout()

In [ ]:
print(N_period)

In [47]:
figure(15,clear='True')
plot(y[:-1],diff(y),'--P')
xlabel('T [K]')
ylabel('H [K/S]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

In [ ]:
figure(16,clear='True')
plot(y[:-1],diff(y)/y[:-1],'--P')
xlabel('T [K]')
ylabel('H/T [K/S /K]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

In [ ]:
len(y)

In [ ]:
figure(17,clear='True')
plot(cloud_size[:-1]/crystal_size,diff(y),'--P')
xlabel('size [m]')
ylabel('H [K/S]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

figure(18,clear='True')
plot(cloud_size[:-1]/crystal_size,diff(y)/y[:-1],'--P')
xlabel('size [m]')
ylabel('H/T [K/S /K]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

# Extract data as npz

In [ ]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

In [ ]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

data_name, num_runs, Time, Temperature \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

# load check
cond = 'DC11_RF18'
outfile = 'Time_and_temp_RFHEAT_N0512_'+str(cond)
with load(outfile+'.npz') as data:
    a = data['time']
    b = data['temp']
print(a)
print(b)
print(shape(a))
print(shape(b))

In [ ]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

In [ ]:
## Data loading (1D arrays)
data_name, num_runs, Time, Temperature \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

In [ ]:
for j,k in enumerate(data_name[0]):
    cond = k[0][52:61]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_N0512_'+str(cond)
    np.savez( outfile, time=Time[j], temp = mean(Temperature[j],axis=0)) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)

In [ ]:
# load check
cond = 'DC11_RF18'
outfile = 'Time_and_temp_RFHEAT_N0512_'+str(cond)
with load(outfile+'.npz') as data:
    a = data['time']
    b = data['temp']
print(a)
print(b)
print(shape(a))
print(shape(b))

In [ ]:
whos

# Work with data

In [ ]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python')
print(file_path)

for j,k in enumerate(file_path):
    cond = k[0][52:61]

In [ ]:
all_Urf = [60,61,62,63,64,65,66,67,68,62.2,62.4,62.6,62.8,
       63.2,63.4,63.6,63.8,40,45,50,55,70,75,80,85]
Urf = []
for j,k in enumerate(file_path):
    str_pos = k.find('N0512_')
    cond = k[str_pos+6:str_pos+15]
    str_pos = cond.find('RF')
    RF_cond = cond[str_pos+2:str_pos+4]
    print(cond,RF_cond)
    Urf.append(all_Urf[int(RF_cond)])
print(sort(Urf))

In [ ]:
# load check
# cond = 'DC11_RF00'
# outfile = 'Time_and_temp_RFHEAT_'+str(cond)
Time = []
Temperature = []
for j,k in enumerate(file_path):
    with load(k) as data:
        Time.append( data['time'] )
        Temperature.append( data['temp'] )
    print(j,k)
print(shape(Time))
print(shape(Temperature))

In [ ]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(10); clf();
i = 11
print(i,Urf[i])
#     subplot(1,9,i+1)
semilogy(Time[i]*1e3,savgol_filter(Temperature[i],31,1),label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
grid()
legend(fontsize=12)

In [ ]:
# plot all temperature curves

imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(1); clf();
for i in range(0,imax-1):
    print(i)
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))
    k_min = k_thr - (k_max - k_thr)
    if k_min < 0:
        k_min = 0

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

#     subplot(1,9,i+1)
    subplot(imax//3,4,i+1)
    semilogy(Time[i][k_min:k_max]*1e3,Temperature[i][k_min:k_max],label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
    grid()
    legend(fontsize=12)
tight_layout()

In [ ]:
# Functions for fit

def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B

def func3(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+abs(t-t0))**D + B

def dfunc2_dt(t, t0, A, B,  C, D):
    return 10**(A*(C + (t - t0)**2)**(-D)*(t - t0) + B)*(-A*D*(C + (t - t0)**2)**(-D)*(t - t0)*(2*t - 2*t0)/(C + (t - t0)**2) + A*(C + (t - t0)**2)**(-D))*log(10)

In [ ]:
# Fit all temperature curves with above function
# savgol_filter used to filter a little bit
# smooth the temperatures that oscillates a bit too much

T_max = 85
T_thr = 1
fit_param = zeros((imax,5))
fit_pcov  = zeros((imax,5,5))
my_order = []
gone_wrong = []

figure(1); clf()
for i in range(0,imax):
    print('<<<',i,'>>>')
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

    k_min = k_thr - int(2*(k_max - k_thr))
    if k_min < 0: k_min = 0
#         print(k_min)

    x  = Time[i][k_min:k_max]
    print(Urf[i])
    print(len(Temperature[i]),k_max,k_thr,k_min)
    try:
        y  = savgol_filter(Temperature[i][k_min:k_max],31,1)
    except:
        y  = Temperature[i][k_min:k_max]
        print('no filter')
        
    x0 = Time[i][k_thr]

    A0  = 1
    B0  = log10(Temperature[i][k_thr])
    C0  = 1.0e-8
    D0  = 0.5
    p0  = array([x0, A0, B0,  C0, D0])
    
    my_order.append(i)

    try:
        popt, pcov = curve_fit(func2, x, log10(y), p0, maxfev= 500000)
        fit_param[i,:] = popt[0], popt[1], popt[2], popt[3], popt[4]
        fit_pcov[i,:]  = pcov

        y_fit = func2(x,popt[0], popt[1], popt[2], popt[3], popt[4])
        y_fit = 10**y_fit

        subplot(6,4,i+1)
        plot(x*1e3, y    ,label=' Vrf = '+str(Urf[i]))
        plot(x*1e3, y_fit,label='Fit',ls='--')
        grid()
        legend(fontsize=10)
        # select cases
        # non zero fit value
    except TypeError:
        print('TypeError for ',Urf[i])
        gone_wrong.append(i)
tight_layout()

In [ ]:
# select only Urf values
# for which non zero fit
# and sort ascending Urf
Y = Urf
X = my_order
my_order = [x for _,x in sorted(zip(Y,X))]
Urf_order = [x for x,_ in sorted(zip(Y,X))]
# delete manually
# other innapropriate values
my_order.remove(17)
Urf_order.remove(75)

print(Urf_order)
print(my_order)

# resort all variables
# in order so
# Urf ascending
imax_order = len(Urf_order)
fit_param_order = zeros((imax_order,5))
fit_pcov_order  = zeros((imax_order,5,5))
Time_order = []
Temperature_order = []
for i,j in enumerate(my_order):
    fit_param_order[i,:] = fit_param[j,:]
    fit_pcov_order[i,:]  = fit_pcov[j,:]
    Time_order.append(Time[j])
    Temperature_order.append(Temperature[j])

In [ ]:
# draw one temp and fit
figure(3); clf()
cm = pylab.get_cmap('jet')

i = 0

print(Urf_order[i])
# print(fit_pcov_order[i,:])
y_fit = func2(Time_order[i],
              fit_param_order[i,0],
              fit_param_order[i,1],
              fit_param_order[i,2],
              fit_param_order[i,3],
              fit_param_order[i,4])
y_fit = 10**y_fit
subplot(121)
plot((Time_order[i])*1e3,Temperature_order[i],color='r')
plot((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
ylabel('T [K]')
grid()

# xlim(3.6,3.7)
# ylim(-1,10)
subplot(122)
semilogy((Time_order[i])*1e3,savgol_filter(Temperature_order[i],31,1),color='r')
semilogy((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
grid()
tight_layout()

In [ ]:
# draw fit parameters
# as a function of Urf
str_labels = ['t0', 'A', 'B', 'C', 'D']
figure('Param'); clf();
for i in range(0,5):
    subplot(1,5,i+1)
    if i == 2:
        plot(Urf_order,10**(fit_param_order[:,i]),'o-', label=str_labels[i])
    else:
        plot(Urf_order,(fit_param_order[:,i]),'o-', label=str_labels[i])
    xlabel('Urf')
    grid()
    legend()

In [ ]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
figure(29,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(savgol_filter(Temperature_order[i],31,1),dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate [K/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [ ]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
coucou = []
figure(31,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    x  = savgol_filter(Temperature_order[i],31,1)
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D) /x
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(x,dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order),marker='+')
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    coucou.append( argmin(abs(x-0.5)) )

#     ax.vlines(x[coucou],1e-5,0.5e0,color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate / T [/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [ ]:
X = []
Y = []
for i,j in enumerate([0,1,13,17]):
#     x = savgol_filter(Temperature_order[i],31,1)
    x  = func2(Time_order[i],t0, A, B, C, D)
    X.append(Urf_order[j])
    Y.append(x[coucou[i]])
print(X,Y)
figure('test',clear='True')
plot(X,Y)

In [ ]:
110/3493 * 15, x[15]

# t0 and T as a function of Urf

In [ ]:
t0    = zeros(imax_order)
y_t0  = zeros(imax_order)
dy_t0 = zeros(imax_order)
for i in range(0, imax_order):
    t0[i], A, B, C, D = fit_param_order[i,:]
    y  = func2(t0[i],t0[i], A, B, C, D);
    y_t0[i]  = 10**y
    dy_t0[i] = dfunc2_dt(t0[i], t0[i], A, B,  C, D)

In [ ]:
def my_lin(x,a,b):
    return a*x+b
def my_power_law(x,a,b):
    return a*(x**-b)

In [ ]:
# log plot
figure(321,clear='True')
ax = subplot(111)
ax.loglog(Urf_order,t0*1e3,ls='-',marker='+',mew=5,ms=10)
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
ax.axis([5*10, 72, 1, 200])
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')

tight_layout()

In [ ]:
figure(322); clf();
ax = subplot(111)
ax.plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order[2:-2],y_t0[2:-2])
ax.plot(Urf_order[2:-2],my_lin(array(Urf_order[2:-2]),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=5))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(64,0.815,f'$T(t_0) = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='left',va='center')

tight_layout()

# identique à 10**B

In [ ]:
figure(323); clf();
subplot(111)
plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

# identique à 10**B

In [ ]:
# check for frequencies